In [20]:
:q!import pandas as pd
import numpy as np
import torch
from torch.utils.data import SubsetRandomSampler, DataLoader

In [13]:
from Notebooks.TorchTextModel import TorchTextModel, TitleDataset

In [15]:
dataset = TitleDataset.TitleDataset()

In [17]:
batch_size = 16
validation_split = .2
random_seed= 42
size = len(dataset)
indices = list(range(size))
split = int(np.floor(validation_split * size))
shuffle = True
if shuffle:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices, test_indices = indices[split:], indices[:int(np.floor(split/2))], indices[int(np.floor(split/2)):split]

In [18]:
train_sampler = SubsetRandomSampler(train_indices)

In [19]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

token_loader = DataLoader(dataset, batch_size=1,
                          sampler=train_sampler)

tokenizer = get_tokenizer('basic_english')
token_iter = token_loader
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(_[0])
vocab = build_vocab_from_iterator(yield_tokens(token_iter))

263lines [00:00, 6961.87lines/s]


In [21]:
vocab_size = len(vocab)
emsize=128
num_class = 2
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = TorchTextModel.TorchTextModel(vocab_size, emsize, num_class).to(device=DEVICE)

In [24]:
saved_vars = torch.load('title_to_continent_torchtext')

In [26]:
model.load_state_dict(saved_vars['model_state_dict'])
model.eval()

TorchTextModel(
  (embedding): EmbeddingBag(383, 128, mode=mean)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)

In [27]:
non_la_art = pd.read_csv('../../data_samples/results/nonLa_art_full.csv')

/Users/mocha/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0,3,6,7,20,21,25,28,33,34,36,43,49,50,51) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [81]:
X = non_la_art.where(non_la_art.title.apply(pd.notna)).dropna(how='all').title

In [88]:
text_pipeline = lambda x: [vocab.__getitem__(tokenizer(y)[0]) if (len(tokenizer(y)) > 0) else 0 for y in x.split(' ')]

In [83]:
is_str_X = X.apply(lambda x: isinstance(x, str))

In [84]:
title_details = pd.DataFrame({'titles': X, 'is_string': is_str_X})

In [85]:
title_details.where(title_details.is_string == False).dropna(how='all')

,titles,is_string


In [89]:
continent = {0: "North America", 1: "South America"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item()

In [90]:
predictions = X.apply(lambda x: continent[predict(x, text_pipeline)])

In [93]:
predictions.to_csv('../../data_samples/results/la_geography_text_class.csv')